In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from final_project.data import read_data
from final_project.preprocessing import (
    compute_future_volatility, compute_returns, 
    compute_squared_returns, add_trading_days,
    add_dates_and_times, add_features_responder,
    write_data, winsorize_predictors,
    add_lagged_metadata
)
from final_project.plotting import (
    plot_hourly_averages, plot_day,
    plot_mses
)

## Load data

In [ ]:
df_btc = read_data("btc")
display(df_btc)

## Compute Responder + Minimal Features

Next, we have to create the responder: 30 minute realized volatility for bitcoin. We also add some basic features: 1 minute, 5 minute, 30 minute, 60 minute, and 2 hour absolute and squared returns.

In [ ]:

df_btc = compute_returns(df_btc, [1, 5, 30, 60, 120])
df_btc = compute_squared_returns(df_btc, [1, 5, 30, 60, 120])
df_btc = add_lagged_metadata(df_btc)
NUM_FEATURES = [f"past_{x}m_{y}ret" for y in ["", "sq_"] for x in [1, 5, 30, 60, 120]] + ["past_30m_quote_volume", "past_30m_trades"]


df_btc = compute_future_volatility(df_btc)
RESPONDER = "future_30m_vol"

df_btc.head()

#### Check distributions

In [ ]:
df_btc[NUM_FEATURES + [RESPONDER]].describe()

Let's look at histograms of 1m and 5m returns, 30m volatility, and lagged metadata:

In [ ]:
sns.histplot(data=df_btc, x="past_1m_ret")

In [ ]:
sns.histplot(data=df_btc, x="past_5m_ret")

It looks like returns should definitely be winsorized, very long tails!

In [ ]:
sns.histplot(data=df_btc, x="future_30m_vol")

This also may benefit from being Winsorized, due to the fat right tail.

In [ ]:
sns.histplot(data=df_btc, x="past_30m_trades")

In [ ]:
sns.histplot(data=df_btc, x="past_30m_quote_volume")

#### Correlations + Relationships
I expect large moves to be correlated with future volatility; large moves are likely followed by large moves, in absolute value. If markets are efficient, returns should not be autocorrelated, but squared returns may be!

In [ ]:
sns.heatmap(df_btc[NUM_FEATURES + [RESPONDER]].corr())

Future volatility seems to be correlated relatively more with longer previous squared returns and shorter absolute returns. This aligns with the hypothesis that the absolute size of the move is what matters.

Let's look at autocorrelation plots next, for the responder, 1 minute returns, and 1 minute squared returns.

In [ ]:
fig = plot_acf(df_btc.iloc[::30]["future_30m_vol"].dropna(), lags=50)

This looks reasonable: volatility is clustered over the span of hours, then the correlation decays, then it increases again
close to 24 hours later! This strongly supports using time of day as a feature.

In [ ]:
fig = plot_acf(df_btc.iloc[::30]["past_1m_ret"].dropna(), lags=50)

Looks like the market is pretty efficient.

In [ ]:
fig = plot_acf(df_btc.iloc[::30]["past_1m_sq_ret"].dropna(), lags=50)

We see a similar pattern to 30m vol here.

#### Preliminary Hypotheses
1. Time of day is an important feature, such as market open.
    - This may break down on non-trading days.
2. Events, such as tariff announcements or economic data release, will impact vol.

In [ ]:
df_btc = add_dates_and_times(df_btc)
fig = plot_hourly_averages(df_btc)

Looks like this should definitely be a feature. Volatility is highest at the beginning of the day when price discovery occurs,
and decays throughout.

Let's see if it's different if we group by NYSE trading/non-trading days.

In [ ]:
df_btc = add_trading_days(df_btc)

# Graph on trading days
mask = df_btc["is_us_trading_day"]
fig = plot_hourly_averages(df_btc[mask])

In [ ]:
# Graph non-trading days
fig = plot_hourly_averages(df_btc[~mask])

Looks like that will be a good responder to include!

Now let's take a look at the announcement of tariffs and a rate cut. Trump's tariffs were announced on April 2, 2025, at around 4 PM ET.

In [ ]:
fig = plot_day(df_btc, "2025-04-02", "future_30m_vol", 30)

It looks like there was some impact; what about a fed rate announcement? We look at October 29, at 2 PM.

In [ ]:
fig = plot_day(df_btc, "2025-10-29", "future_30m_vol", 30)

Big spike!!! How about economic indicators? Now we look at the US CPI, released April 10 at 8:30 ET

In [ ]:
fig = plot_day(df_btc, "2025-04-10", "future_30m_vol", 30)

There is definitely a spike at 8:30, then another one at market open. So we should add features for economic data release and fed events.

#### Single Predictor for Responder
I have a hypothesis that some moving average of volatility will be the best predictor for forward looking vol - there is a tradeoff between incorporating new information and having a stable predictor. We will use a continuum of exponentially weighted moving averages of the mean of volatility, as well as a simple rolling mean, and see which have the lowest MSE if used as a baseline predictor!

In [ ]:
fig = plot_mses(df_btc, 200)

Cool - we will add a feature for ewm with span 50 (close to 53!)

## Data Cleaning + Feature Addition
Above we saw that we should add time of day, trading calendars, economics/fed data releases, and recent volatility as features. 

In fact, I will add trading days for UK, European, and Asian exchanges too. Also, I'll add daylight savings flags for each of these markets.

We will also add all of the returns/squared returns data from before (slight risk of overfitting, can select features later). Finally, I'll add past 5 minute vol as a percentage of past 30m vol (increasing or decreasing??), and past 30m vol / past 60m vol.

It also seems reasonable to add some longer realized vol measures, like 1 day, 1 week, and 1 month.

#### Add features and responders

In [ ]:
df_raw = read_data("btc")
df = add_features_responder(df_raw)
df.head()

#### Clip data
Here, I will clip my numerical features to avoid having super high-leverage points on outliers. The data is quite clean in general, though.

I will use percentiles from the first 6 months of 2024 to avoid lookahead when we change to the test set.

In [ ]:
# Write unclipped data to parquet
write_data(df, "clean_data")

df_clipped = winsorize_predictors(df)
write_data(df_clipped, "clean_data_clipped")

The prepare_data script can also do the steps above!

Let's take a quick look at correlation between features.

In [ ]:
from final_project.preprocessing import NUM_FEATURES
sns.heatmap(df[NUM_FEATURES].corr())